In [1]:
import urllib
import json
import pandas as pd

In [22]:
account_step2_df = pd.read_csv('account_step2_df.csv')
account_step2_df[:1]

,corp_name,corp_code,bsns_year,reprt_code,data
0,LG파워콤,318617,2020,11011,"{""status"":""013"",""message"":""조회된 데이타가 없습니다.""}"


In [23]:
account_df = pd.DataFrame(columns=["corp_name","bsns_year","reprt_code", "자산총계","부채총계","자본총계","매출액","영업이익","당기순이익"])

for index, row in account_step2_df.iterrows():
    data = json.loads(row["data"])
    if data['status'] != '000':
        continue
    
    new_row = {
        "corp_name": row["corp_name"],
        "bsns_year": row["bsns_year"],
        "reprt_code": row["reprt_code"],
    }
    
    for item in data['list']:
        if item["account_nm"] == "자산총계" and item["fs_div"] == "OFS":
            new_row["자산총계"] = item["thstrm_amount"]
            
    for item in data['list']:
        if item["account_nm"] == "부채총계" and item["fs_div"] == "OFS":
            new_row["부채총계"] = item["thstrm_amount"]
            
    for item in data['list']:
        if item["account_nm"] == "자본총계" and item["fs_div"] == "OFS":
            new_row["자본총계"] = item["thstrm_amount"]
            
    for item in data['list']:
        if item["account_nm"] == "매출액" and item["fs_div"] == "OFS":
            new_row["매출액"] = item["thstrm_amount"]
            
    for item in data['list']:
        if item["account_nm"] == "영업이익" and item["fs_div"] == "OFS":
            new_row["영업이익"] = item["thstrm_amount"]
            
    for item in data['list']:
        if item["account_nm"] == "당기순이익" and item["fs_div"] == "OFS":
            new_row["당기순이익"] = item["thstrm_amount"]
    
    account_df = account_df.append(new_row, ignore_index=True)


account_df.to_csv('account_step3_df.csv', index=False)
account_df[account_df['corp_name'].str.contains("LG화학")]


,corp_name,bsns_year,reprt_code,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
60,LG화학,2020,11012,"29,024,574,000,000","12,209,773,000,000","16,814,801,000,000","5,287,415,000,000","315,067,000,000","193,992,000,000"
61,LG화학,2019,11011,"27,027,086,000,000","10,566,856,000,000","16,460,230,000,000","22,312,561,000,000","582,425,000,000","365,924,000,000"
62,LG화학,2019,11012,"26,569,733,000,000","10,049,761,000,000","16,519,972,000,000","5,576,488,000,000","267,982,000,000","223,612,000,000"
63,LG화학,2018,11011,"23,730,578,000,000","7,180,672,000,000","16,549,906,000,000","23,315,318,000,000","1,953,618,000,000","1,581,064,000,000"
64,LG화학,2018,11012,"22,816,383,000,000","6,850,225,000,000","15,966,158,000,000","5,810,297,000,000","595,891,000,000","496,659,000,000"
65,LG화학,2017,11011,"21,148,136,000,000","5,642,604,000,000","15,505,532,000,000","20,880,236,000,000","2,293,229,000,000","1,605,223,000,000"
66,LG화학,2017,11012,"18,887,473,000,000","4,032,647,000,000","14,854,826,000,000","5,154,792,000,000","599,352,000,000","544,079,000,000"
67,LG화학,2016,11011,"16,775,894,000,000","3,299,563,000,000","13,476,331,000,000","17,264,845,000,000","1,813,245,000,000","1,379,589,000,000"
68,LG화학,2016,11012,"16,626,149,000,000","3,701,218,000,000","12,924,931,000,000","4,309,410,000,000","558,004,000,000","489,064,000,000"
69,LG화학,2015,11011,"16,231,596,000,000","3,787,033,000,000","12,444,563,000,000","17,334,109,000,000","1,620,092,000,000","1,203,855,000,000"
